# Automatische Annotation von Texten

## Requirements
- trafilatura (https://trafilatura.readthedocs.io/en/latest/)
- Stanza (https://stanfordnlp.github.io/stanza/index.html)
- SoMaJo (https://github.com/tsproisl/SoMaJo) und SoMeWeTa (https://github.com/tsproisl/SoMeWeTa)
- spaCy (https://spacy.io/usage)

## Module importieren

In [1]:
import trafilatura
import stanza
import somajo
import someweta
import spacy


## Vorverarbeitung der Texte

In [2]:
downloaded = trafilatura.fetch_url("https://en.wikipedia.org/wiki/Computational_linguistics")
cl = trafilatura.extract(downloaded)

In [3]:
print(cl)

Computational linguistics
|Part of a series on|
|Linguistics|
|Portal|
Computational linguistics is an interdisciplinary field concerned with the computational modelling of natural language, as well as the study of appropriate computational approaches to linguistic questions. In general, computational linguistics draws upon linguistics, computer science, artificial intelligence, mathematics, logic, philosophy, cognitive science, cognitive psychology, psycholinguistics, anthropology and neuroscience, among others.
[edit]
Traditionally, computational linguistics emerged as an area of artificial intelligence performed by computer scientists who had specialized in the application of computers to the processing of a natural language. With the formation of the Association for Computational Linguistics (ACL)[1] and the establishment of independent conference series, the field consolidated during the 1970s and 1980s.
The Association for Computational Linguistics defines computational linguisti

In [4]:
paragraphs = list()
for line in cl.split("\n"):
    if not (line.startswith("|") or line.startswith("- ^")) and len(line) > 50:
        paragraphs.append(line)

print(len(paragraphs))
print(paragraphs[0])

64
Computational linguistics is an interdisciplinary field concerned with the computational modelling of natural language, as well as the study of appropriate computational approaches to linguistic questions. In general, computational linguistics draws upon linguistics, computer science, artificial intelligence, mathematics, logic, philosophy, cognitive science, cognitive psychology, psycholinguistics, anthropology and neuroscience, among others.


## Stanza
### Sprachmodelle und Pipelines
Um Stanza verwenden zu können, benötigt man erst eine Pipeline für die Sprache, die man verarbeiten möchte. Eine Pipeline enthält verschiedene Elemente, z.B. einen Tokenisierer, einen Part-of-Speech-Tagger usw.

Wenn man außer der gewünschten Sprache keine weiteren Parameter angibt, lädt Stanza netterweise eine Standard-Pipeline. Um ein Sprachmodell herunterzuladen, führt man zunächst `stanza.download()` aus – wenn man das schon einmal getan hat, braucht man die Zeile nicht mehr im Skript, da sich das Modell dann bereits auf der Festplatte befindet.

Was genau für ein Sprachmodell man herunterlädt, lässt sich spezifizieren.
- Eine Einführung in die Möglichkeiten gibt es hier: https://stanfordnlp.github.io/stanza/installation_usage.html#building-a-pipeline.
- Eine Übersicht über die aktuell verfügbaren Modelle und Parameter von `stanza.download()` gibt es hier: https://stanfordnlp.github.io/stanza/models.html

Es ist wichtig, Modelle auszuwählen, die nicht nur zur Sprache unserer Daten passen, sondern möglichst auch auf vergleichbaren Daten trainiert wurden. Wenn man bspw. Tweets mit einem Modell annotieren möchte, das ausschließlich auf Zeitungstexten oder Romanen des 19. Jahrhunderts trainiert wurde (oder umgekehrt), ist die Genauigkeit der Annotation wahrscheinlich nicht allzu hoch.

In [5]:
stanza.download("en")

2022-09-14 17:01:54 INFO: Downloading default packages for language: en (English)...
2022-09-14 17:01:55 INFO: File exists: /home/ausgerechnet/stanza_resources/en/default.zip
2022-09-14 17:01:59 INFO: Finished downloading models and saved to /home/ausgerechnet/stanza_resources.


Jetzt kann die Pipeline gebaut werden. Dabei sollten wir gut überlegen, ob wir wirklich alles brauchen, denn einiges braucht Prozessorleistung und Arbeitsspeicher ... Ohne GPU ist die NER sehr langsam, der Phrasenstruktur-Parser (constituency) extrem langsam!

In [6]:
pipeline = stanza.Pipeline("en")
# pipeline = stanza.Pipeline("en", processors="tokenize,pos,lemma")

2022-09-14 17:02:00 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-09-14 17:02:00 INFO: Use device: cpu
2022-09-14 17:02:00 INFO: Loading: tokenize
2022-09-14 17:02:00 INFO: Loading: pos
2022-09-14 17:02:00 INFO: Loading: lemma
2022-09-14 17:02:00 INFO: Loading: depparse
2022-09-14 17:02:00 INFO: Loading: sentiment
2022-09-14 17:02:01 INFO: Loading: constituency
2022-09-14 17:02:01 INFO: Loading: ner
2022-09-14 17:02:01 INFO: Done loading processors!


Die Anwendung der Pipeline gestaltet sich nun denkbar einfach. Ohne GPU ist sie aber mitunter extrem langsam, wobei einige Internet-Quellen empfehlen, die Umgebungsvariable `OMP_NUM_THREADS` entsprechend zu setzen (was aber nicht immer zum Erfolg führt).

In [7]:
%env OMP_NUM_THREADS=4
# takes a couple of minutes on my X1 Carbon
doc = pipeline("\n".join(paragraphs))

print(type(doc))

env: OMP_NUM_THREADS=4
<class 'stanza.models.common.doc.Document'>


### Output und Konvertierung in andere Formate

Der Output der Pipeline hat die Klasse `Document` und bietet verschiedene Methoden, um auf Sätze, Token und Annotationen zuzugreifen (siehe https://stanfordnlp.github.io/stanza/data_objects.html). Hier einige Beispiele:

In [8]:
print(doc.num_tokens)
print(len(doc.sentences))

4501
159


In [9]:
sent = doc.sentences[12] # sehen wir uns den 1. Satz an
for tok in sent.words:
    print(f"{tok.id:2d} {tok.text:16s} {tok.pos:8s} {tok.xpos:8s} {tok.lemma}")

 1 Theoretical      ADJ      JJ       theoretical
 2 computational    ADJ      JJ       computational
 3 linguistics      NOUN     NN       linguistics
 4 includes         VERB     VBZ      include
 5 the              DET      DT       the
 6 development      NOUN     NN       development
 7 of               ADP      IN       of
 8 formal           ADJ      JJ       formal
 9 theories         NOUN     NNS      theory
10 of               ADP      IN       of
11 grammar          NOUN     NN       grammar
12 (                PUNCT    -LRB-    (
13 parsing          NOUN     NN       parsing
14 )                PUNCT    -RRB-    )
15 and              CCONJ    CC       and
16 semantics        NOUN     NN       semantics
17 ,                PUNCT    ,        ,
18 often            ADV      RB       often
19 grounded         VERB     VBN      ground
20 in               ADP      IN       in
21 formal           ADJ      JJ       formal
22 logics           NOUN     NNS      logic
23 and           

In [10]:
print(sent.sentiment)

1


In [11]:
for i in range(len(doc.sentences)):
    print(doc.sentences[i].sentiment, ":", " ".join([tok.text for tok in doc.sentences[i].words]))

1 : Computational linguistics is an interdisciplinary field concerned with the computational modelling of natural language , as well as the study of appropriate computational approaches to linguistic questions .
1 : In general , computational linguistics draws upon linguistics , computer science , artificial intelligence , mathematics , logic , philosophy , cognitive science , cognitive psychology , psycholinguistics , anthropology and neuroscience , among others .
1 : Traditionally , computational linguistics emerged as an area of artificial intelligence performed by computer scientists who had specialized in the application of computers to the processing of a natural language .
1 : With the formation of the Association for Computational Linguistics ( ACL ) [ 1 ] and the establishment of independent conference series , the field consolidated during the 1970s and 1980s .
1 : The Association for Computational Linguistics defines computational linguistics as : ... the scientific study of

In [12]:
print(sent.constituency) # parse tree
for dep in sent.dependencies:
    l1 = dep[0].lemma or ""
    rel = dep[1]
    l2 = dep[2].lemma
    print(f"{l1:>10s}  --{rel:5}->  {l2:10s}")

(ROOT (S (NP (JJ Theoretical) (JJ computational) (NN linguistics)) (VP (VBZ includes) (NP (NP (DT the) (NN development)) (PP (IN of) (NP (NP (NP (JJ formal) (NNS theories)) (PP (IN of) (NP (NN grammar) (-LRB- -LRB-) (NN parsing) (-RRB- -RRB-) (CC and) (NN semantics)))) (, ,) (VP (ADVP (RB often)) (VBN grounded) (PP (IN in) (NP (NP (JJ formal) (NNS logics)) (CC and) (NP (JJ symbolic) (-LRB- -LRB-) (ADJP (NN knowledge) (HYPH -) (VBN based)) (-RRB- -RRB-) (NNS approaches))))))))) (. .)))
linguistics  --amod ->  theoretical
linguistics  --amod ->  computational
   include  --nsubj->  linguistics
            --root ->  include   
development  --det  ->  the       
   include  --obj  ->  development
    theory  --case ->  of        
    theory  --amod ->  formal    
development  --nmod ->  theory    
   grammar  --case ->  of        
    theory  --nmod ->  grammar   
   parsing  --punct->  (         
    theory  --appos->  parsing   
   parsing  --punct->  )         
 semantics  --cc   ->  a

In [13]:
[(f"{x.text} ({x.type})") for x in doc.entities[0:20]]

['the Association for Computational Linguistics (ORG)',
 'ACL (ORG)',
 '1 (CARDINAL)',
 '1970s (DATE)',
 '1980s (DATE)',
 'The Association for Computational Linguistics (ORG)',
 '2020 (DATE)',
 'the 2000s (DATE)',
 'NLP (ORG)',
 'ACL (ORG)',
 '3 (CARDINAL)',
 'the mid-2010s (DATE)',
 'Socher et al. (ORG)',
 '2012 (DATE)',
 '5 (CARDINAL)',
 'Deep Learning (ORG)',
 'the ACL 2012 (DATE)',
 '2015 (DATE)',
 'NLP (ORG)',
 'Natural (WORK_OF_ART)']

Mit `to_dict()` können wir das Dokument in eine Liste von Listen von Dictionaries (für jedes Token in jedem Satz) konvertieren. Mit `CoNLL.convert_dict()` können wir diese wiederum ins [CoNLL-Format](https://universaldependencies.org/format.html) überführen, was für den Output vielleicht schöner ist:

In [14]:
conll = stanza.utils.conll.CoNLL.convert_dict(doc.to_dict())
for sentence in conll[0:3]:
    for token in sentence:
        print("\t".join(token))

1	Computational	Computational	ADJ	JJ	Degree=Pos	2	amod	_	start_char=0|end_char=13|ner=O
2	linguistics	linguistics	NOUN	NN	Number=Sing	6	nsubj	_	start_char=14|end_char=25|ner=O
3	is	be	AUX	VBZ	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	6	cop	_	start_char=26|end_char=28|ner=O
4	an	a	DET	DT	Definite=Ind|PronType=Art	6	det	_	start_char=29|end_char=31|ner=O
5	interdisciplinary	interdisciplinary	ADJ	JJ	Degree=Pos	6	amod	_	start_char=32|end_char=49|ner=O
6	field	field	NOUN	NN	Number=Sing	0	root	_	start_char=50|end_char=55|ner=O
7	concerned	concern	VERB	VBN	Tense=Past|VerbForm=Part	6	acl	_	start_char=56|end_char=65|ner=O
8	with	with	ADP	IN	_	11	case	_	start_char=66|end_char=70|ner=O
9	the	the	DET	DT	Definite=Def|PronType=Art	11	det	_	start_char=71|end_char=74|ner=O
10	computational	computational	ADJ	JJ	Degree=Pos	11	amod	_	start_char=75|end_char=88|ner=O
11	modelling	modelling	NOUN	NN	Number=Sing	7	obl	_	start_char=89|end_char=98|ner=O
12	of	of	ADP	IN	_	14	case	_	start_char=99|end_c

im CoNLL-Format in Datei speichern:

In [15]:
stanza.utils.conll.CoNLL.write_doc2conll(doc, "../data/cl-stanza.conllu")

## SoMaJo & SoMeWeTa

SoMaJo benötigt keine weiteren Dateien. SoMeWeTa braucht ein vortrainiertes Sprachmodell. Fürs Englische stehen uns zwei zur Auswahl (eins auf der GitHub-Seite, eins, das auch auf Web-Texten trainiert wurde, auf StudOn).

Für SoMaJo initialisiert man den Tagger folgendermaßen:

In [ ]:
model = "../data/english_newspaper_2017-09-15.model"  # needs this file in this directory, d'uh
asptagger = someweta.ASPTagger()
asptagger.load(model)  # takes some time

Für den Tokenisierer wird zunächst ein `SoMaJo`-Objekt erzeugt, das folgende Funktionen bereitstellt: `tokenize_text`, `tokenize_text_file`, `tokenize_xml` und `tokenize_xml_file`.

In [ ]:
tokenizer = somajo.SoMaJo(language="en_PTB")

Zum Tokenisieren wählen wir `tokenize_text()`, da unser Text bereits als Liste von Absätzen vorliegt.

In [ ]:
sentences = list(tokenizer.tokenize_text(paragraphs))
len(sentences)

In [ ]:
[token.text for token in sentences[0]]

### Output
Das CoNLL-Format lohnt sich hier wahrscheinlich nicht (könnten wir natürlich trotzdem wählen, wir müssten bloß in viele Felder `_` schreiben). Wir belassen es mal bei der Angabe von Token und Wortart, jeweils tab-separiert auf einer Zeile. Zwischen Sätzen fügen wir Leerzeilen ein.

In [ ]:
for sentence in sentences:
    tokens = [token.text for token in sentence]
    tagged_sentence = asptagger.tag_sentence(tokens)
    print("\n".join("\t".join(t) for t in tagged_sentence), "\n", sep="")

In [ ]:
import gzip
with gzip.open("../data/cl-somajo-someweta.vrt.gz", "wt") as f:
    f.write("<text>\n")
    for sentence in sentences:
        tokens = [token.text for token in sentence]
        tagged_sentence = asptagger.tag_sentence(tokens)
        f.write("<s>\n")
        f.write("\n".join("\t".join(t) for t in tagged_sentence) + "\n")
        f.write("</s>\n")
    f.write("</text>\n")

### spaCy

Vortrainierte spaCy-Modelle sind für viele Sprachen verfügbar (siehe https://spacy.io/models). Im Gegensatz zu Stanza und SoMeWeTa werden die Modelle als normale pip-Pakete installiert und nicht automatisch ins Home-Verzeichnis des Nutzers heruntergeladen. Daher muss die Installation auf der Kommandozeile vorgenommen werden.  Wir verwenden hier ein relativ kleines, für CPU optimiertes Modell mit word embeddings ([en_core_web_md](https://spacy.io/models/en#en_core_web_md)).

```
python3.9 -m spacy download en_core_web_sm
```

Wir können nun das spaCy-Paket und dieses Modell laden:

In [ ]:
import spacy
pipeline2 = spacy.load("en_core_web_sm")
print(pipeline2.component_names)

`pipeline2` führt die komplette Pipeline mit allen enthaltenen Komponenten aus. Beim Aufruf können nicht benötigte Komponenten mit dem Argument `disable` übersprungen werden.

In [ ]:
# doc2 = pipeline2(test, disable=["lemmatizer", "ner"])
doc2 = pipeline2("\n".join(paragraphs))

### Output und Visualisierung

Nun kann über die einzelnen Token des Textes komplett oder satzweise iteriert werden:

In [ ]:
for sent in list(doc2.sents)[0:2]:
    print("# " + sent.text)
    for tok in sent:
        print(tok.i, tok.orth_, tok.lemma_, tok.pos_, tok.tag_, tok.ent_type_,
              tok.dep_, tok.head.lemma_, tok.shape_, "OOV" if tok.is_oov else "_", sep="\t")
    print()

Alle _noun chunks_ (minimale Nominalphrasen) und Entitäten (_named entitites_) im Text:

In [ ]:
print(list(doc2.noun_chunks)[0:10])  # basic noun chunks

In [ ]:
for ne in doc2.ents[0:10]:
    print(f"{ne.text} [{ne.label_}]")

Entitäten und die Dependenz-Parses können auch direkt mit spaCy **visualisiert** werden. Das `displacy`-Modul erkennt dabei automatisch, dass es in einem Jupyter-Notebook ausgeführt wird und liefert ein passendes Ausgabeformat.

In [ ]:
from spacy import displacy
displacy.render(list(doc2.sents)[12:18], style="ent")

In [ ]:
render_style = {'compact': False, 'distance': 120, 'word_spacing': 20, 'arrow_spacing': 10}
displacy.render(list(doc2.sents)[0], style="dep", options=render_style)